In [12]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [13]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [14]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

In [15]:
indice = parametros.gestion_mintic_concat_index

### Función para generar JSON compatible con ES

In [16]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

Se usan dos fechas control:
* jerarquia_gestion_totales_incidentes
* jerarquia_gestion_totales_llamadas

In [17]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_totales_incidentes"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['gestion.fechaControl']
except Exception as e:
    print(e.message)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-05-15T00:00:00


In [18]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_totales_llamadas"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion2 = doc["_source"]['gestion.fechaControl']
except Exception as e:
    print(e.message)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion2 = '2021-04-01T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion2)

ultima fecha para control de ejecucion: 2021-05-05T00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, entre otros)

In [22]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source":  ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion']            
            
        },
        size=total_docs
    )
    #print(es.info())  se agregaron mas campos 01/07/21
    #['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
    #                       ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede'
    #                       ,'DDA','estadoInstalacion','nombreInstitucionEd','matricula','municipioPDET']
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e.message)
    print("fecha:",now,"- Error en lectura de datos semilla")
    exit()




In [31]:
datos_semilla['matricula']=datos_semilla['matricula'].apply(lambda x:x if x!='No aplica' else 0)

In [32]:
# datos_semilla[datos_semilla['matricula']=='No aplica'][['site_id','matricula']]

,site_id,matricula


* se valida latitud longitud
* Se crea campo location
* se renombran campos de semilla

In [ ]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.totales.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.totales.location']=datos_semilla['gestion.totales.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

#datos_semilla

### Leyendo indice servicemanager-incidentes

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* clr_bmc_location, es la llave para cruzar con cada centro de conexión.
* product_type, es el detalle del ticket.
* clr_bmc_host, estado la IP
* assignment, es el sujeto asignado al ticket
* contact_name, usuario del ticket
* open_time y resolved_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* number, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* category y subcategory, clasificacion dada por mintic

In [ ]:
def traeSMIncidentes(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.sm_incidentes,
        body={
            "_source": ["clr_bmc_location","severity","open_time"]
            ,"query": {
                "range": {
                  "@timestamp": {
                  #"open_time": {
                    "gte": fecha_max,
                    "lt": fecha_tope
                  }
                }
            }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    

Realizando bucle hasta conseguir datos de servicemanager-incidentes o hasta la fecha actual para realizar la carga de datos

In [ ]:
def traeSMInteracciones(fecha_max,fecha_tope):
    total_docs = 10000    
    response = es.search(
        index= parametros.sm_interaciones,
        body={
            "_source": ["clr_txt_idbneficmtc","variable1","source","open_time","category"]
            ,"query": {
                "bool":{
                    "must_not": [
                        {
                          "term": {
                            "clr_txt_idbneficmtc": "null"
                          }
                        }
                      ],
                      "filter": {
                        "range": {
                            #"@timestamp": {
                             "open_time": {
                                "gte": fecha_max,
                                "lt": fecha_tope
                            }
                        }
                      }
                 }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

In [ ]:
#fecha_ejecucion = "2021-04-01T12:40:00"
#print(" f1: ", fecha_max_mintic, "  ***** f2: ", fecha_tope_mintic)
#print(" fecha_ejecucion : ",fecha_ejecucion, " fecha_ejecucion2 : ",fecha_ejecucion2)
#fecha_ejecucion = "2021-06-12T12:40:00"

* Se calcula rango en base a la fecha de ejecución. Para este caso es de 24 horas.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [ ]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
#fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(days=124)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")

datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)

if (datos_SM_incidents is None or datos_SM_incidents.empty):
    while (datos_SM_incidents is None or datos_SM_incidents.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)
else:
    pass
#datos_SM_incidents   

In [ ]:
fecha_max_mintic2 = fecha_ejecucion2
fecha_tope_mintic2 = (datetime.strptime(fecha_max_mintic2, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")

datos_SM_interactions = traeSMInteracciones(fecha_max_mintic2,fecha_tope_mintic2)

if datos_SM_interactions is None or datos_SM_interactions.empty:
    while (datos_SM_interactions is None or datos_SM_interactions.empty) and ((datetime.strptime(fecha_max_mintic2[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic2 = (datetime.strptime(fecha_max_mintic2, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic2 = (datetime.strptime(fecha_tope_mintic2, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_interactions = traeSMInteracciones(fecha_max_mintic2,fecha_tope_mintic2)
else:
    pass
#datos_SM_interactions

# De aqui hacia arriba esta OK!

### Se calcula conteo de incidentes

* Se calcula fecha control
* Se totaliza incidentes por site_id y fecha control
* Se hace merge entre semilla e incidentes

In [ ]:
try:
    datos_SM_incidents['gestion.fechaControl'] = datos_SM_incidents['open_time'].str.split(".", n = 1, expand = True)[0].str[0:-4] + '0:00'
    datos_SM_incidents['gestion.fechaControl'] = datos_SM_incidents['gestion.fechaControl'].str.replace("T"," ")
    datos_SM_incidents = datos_SM_incidents.groupby(['clr_bmc_location','gestion.fechaControl']).count().reset_index()
    datos_SM_incidents = datos_SM_incidents.rename(columns={'clr_bmc_location':'site_id', 'severity':'gestion.totales.total_incidentes'})
    conteo_incidentes = pd.merge(datos_SM_incidents,datos_semilla, on=['site_id'],how='inner')
    conteo_incidentes = conteo_incidentes.rename(columns={'id_Beneficiario' : 'gestion.totales.id_Beneficiario'
#                                                           ,'nombreSede':'gestion.totales.nombreSede'
                                                          ,'site_id':'gestion.totales.site_id'
                                                          ,'nombre_departamento':'gestion.totales.dptoGestion'
                                                          ,'nombre_municipio':'gestion.totales.muniGestion'
                                                          ,'nombre_centro_pob':'gestion.totales.nombre_centro_pob'
                                                          ,'COD_ISO':'gestion.totales.COD_ISO'
                                                          ,'codDanesede':'gestion.totales.codDanesede'
                                                          ,'energiadesc':'gestion.totales.energiadesc'
                                                          ,'DDA':'gestion.totales.DDA'
                                                          ,'estadoInstalacion':'gestion.totales.estadoInstalacion'
                                                          ,'nombreSede':'gestion.totales.nombreInstitucionEd'
                                                          ,'matricula':'gestion.totales.matricula'
                                                          ,'cod_servicio': 'gestion.estado.cod_servicio'
                                                          ,'codDaneMuni':'gestion.estado.codDaneMuni'
#                                                           ,'municipioPDET':'gestion.totales.municipioPDET'
                                                          
                                                                                                                          ,'codDaneMuni':'gestion.estado.codDaneMuni'
                                                        ,'codCentroPoblado':'gestion.estado.codCentroPoblado'
                                                        ,'codDaneInstitucionEdu':'gestion.estado.codDaneInstitucionEdu'
                                                        ,'tipoSitio':'gestion.estado.tipoSitio'
                                                        ,'detalleSitio':'gestion.estado.detalleSitio'
                                                        ,'energia':'gestion.estado.energia'
                                                        ,'region':'gestion.estado.region'
                                                        ,'grupoDesc' :'gestion.estado.grupoDesc'
                                                          
                                                         })
    
    conteo_incidentes['gestion.totales.nombreSede'] = conteo_incidentes['gestion.totales.nombreInstitucionEd']
    conteo_incidentes['gestion.estado.estadoInstalacion'] = conteo_incidentes['gestion.totales.estadoInstalacion']
    
    conteo_incidentes["gestion.fecha"] = conteo_incidentes["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    conteo_incidentes["gestion.hora"] = conteo_incidentes["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    conteo_incidentes["gestion.anyo"] = conteo_incidentes["gestion.fecha"].str[0:4]
    conteo_incidentes["gestion.mes"] = conteo_incidentes["gestion.fecha"].str[5:7]
    conteo_incidentes["gestion.dia"] = conteo_incidentes["gestion.fecha"].str[8:10]
except:
    conteo_incidentes = pd.DataFrame()
#conteo_incidentes

### Se calcula conteo de llamadas correos

* Se calcula fecha control
* Se totaliza source por site_id y fecha control
* Se hace merge entre semilla e incidentes
* Se separa el conteo de llamadas y correos

In [ ]:
try:
    datos_SM_interactions['gestion.fechaControl'] = datos_SM_interactions['open_time'].str.split(".", n = 1, expand = True)[0].str[0:-4] + '0:00'
    datos_SM_interactions['gestion.fechaControl'] = datos_SM_interactions['gestion.fechaControl'].str.replace("T"," ")
    datos_SM_interactions['gestion.fechaControl'].str[0:-4] + '0:00'
    datos_SM_interactions = datos_SM_interactions.rename(columns={'variable1' : 'gestion.totales.llamadas','clr_txt_idbneficmtc':'id_Beneficiario'})
    datos_SM_interactions = datos_SM_interactions.groupby(['id_Beneficiario','source','gestion.fechaControl']).count().reset_index()           
    datos_SM_interactions['id_Beneficiario'] = datos_SM_interactions['id_Beneficiario'].astype(int)
    #datos_SM_interactions['gestion.tiempoRespuesta'] = datos_SM_interactions.apply(lambda row: row.close_time - row.open_time, axis=1)
    conteo_interacciones = pd.merge(datos_SM_interactions,datos_semilla, on=['id_Beneficiario'],how='inner')
    conteo_interacciones = conteo_interacciones.rename(columns={'id_Beneficiario' : 'gestion.totales.id_Beneficiario'
#                                                                 ,'nombreSede':'gestion.totales.nombreSede'
                                                                ,'site_id':'gestion.totales.site_id'
                                                                ,'nombre_departamento':'gestion.totales.dptoGestion'
                                                                ,'nombre_municipio':'gestion.totales.muniGestion'
                                                                ,'nombre_centro_pob':'gestion.totales.nombre_centro_pob'
                                                                ,'COD_ISO':'gestion.totales.COD_ISO'
                                                                ,'codDanesede':'gestion.totales.codDanesede'
                                                                ,'energiadesc':'gestion.totales.energiadesc'
                                                                ,'DDA':'gestion.totales.DDA'
                                                                ,'estadoInstalacion':'gestion.totales.estadoInstalacion'
                                                                ,'nombreSede':'gestion.totales.nombreInstitucionEd'
                                                                ,'matricula':'gestion.totales.matricula'
                                                                ,'cod_servicio': 'gestion.estado.cod_servicio'
                                                                ,'codDaneMuni':'gestion.estado.codDaneMuni'
#                                                                 ,'municipioPDET':'gestion.totales.municipioPDET'
                                                                
        
                                                                ,'codDaneMuni':'gestion.estado.codDaneMuni'
                                                                ,'codCentroPoblado':'gestion.estado.codCentroPoblado'
                                                                ,'codDaneInstitucionEdu':'gestion.estado.codDaneInstitucionEdu'
                                                                ,'tipoSitio':'gestion.estado.tipoSitio'
                                                                ,'detalleSitio':'gestion.estado.detalleSitio'
                                                                ,'energia':'gestion.estado.energia'
                                                                ,'region':'gestion.estado.region'
                                                                ,'grupoDesc' :'gestion.estado.grupoDesc'
                                                                
                                                               })
    
    conteo_interacciones['gestion.totales.nombreSede'] = conteo_interacciones['gestion.totales.nombreInstitucionEd']
    conteo_interacciones['gestion.estado.estadoInstalacion'] = conteo_interacciones['gestion.totales.estadoInstalacion']
    
    conteo_interacciones["gestion.fecha"] = conteo_interacciones["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    conteo_interacciones["gestion.hora"] = conteo_interacciones["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    conteo_interacciones["gestion.anyo"] = conteo_interacciones["gestion.fecha"].str[0:4]
    conteo_interacciones["gestion.mes"] = conteo_interacciones["gestion.fecha"].str[5:7]
    conteo_interacciones["gestion.dia"] = conteo_interacciones["gestion.fecha"].str[8:10]
    
    conteo_llamadas = conteo_interacciones[conteo_interacciones['source'] == "6"]
    conteo_correos = conteo_interacciones[conteo_interacciones['source'] == "5"]
except Exception as e:
    print(e)
    conteo_llamadas = pd.DataFrame()
    conteo_correos = pd.DataFrame()
    

# calculo del total de ticket 


In [ ]:
try:
    
    #calculo del total de ticket : que seria el total de (conteo_interacciones  + conteo_incidentes)

    #prueba =conteo_incidentes
    #conteo_interacciones
    
    if not (isinstance(conteo_incidentes, pd.core.frame.DataFrame) and   \
            isinstance(conteo_interacciones, pd.core.frame.DataFrame) ):
        raise Exception()
        
    tot = pd.concat( [conteo_incidentes, conteo_interacciones] )
    #tot = pd.concat( [conteo_incidentes, prueba] )


    tot = tot[['gestion.totales.id_Beneficiario','gestion.totales.site_id','gestion.fechaControl']]
    tot["gestion.totales.total_tickets"] = 0

    totalesx = tot.groupby(['gestion.totales.id_Beneficiario','gestion.totales.site_id','gestion.fechaControl']).agg(['count']).reset_index()

    totalesx.columns =totalesx.columns.droplevel(1)  
    
    totalesx['gestion.fechaControl'] = totalesx['gestion.fechaControl'].str.replace("T"," ")
    totalesx['gestion.fechaControl'].str[0:-4] + '0:00'
    
    totalesx = totalesx.rename(columns={'gestion.totales.id_Beneficiario' : 'id_Beneficiario'})
    
    totalesx['id_Beneficiario'] = totalesx['id_Beneficiario'].astype(int)
    
    conteo_totalesx = pd.merge(totalesx,datos_semilla, on=['id_Beneficiario'],how='inner')
    conteo_totalesx = conteo_totalesx.rename(columns={'id_Beneficiario' : 'gestion.totales.id_Beneficiario'
#                                                                 ,'nombreSede':'gestion.totales.nombreSede'
                                                                ,'site_id':'gestion.totales.site_id'
                                                                ,'nombre_departamento':'gestion.totales.dptoGestion'
                                                                ,'nombre_municipio':'gestion.totales.muniGestion'
                                                                ,'nombre_centro_pob':'gestion.totales.nombre_centro_pob'
                                                                ,'COD_ISO':'gestion.totales.COD_ISO'
                                                                ,'codDanesede':'gestion.totales.codDanesede'
                                                                ,'energiadesc':'gestion.totales.energiadesc'
                                                                ,'DDA':'gestion.totales.DDA'
                                                                ,'estadoInstalacion':'gestion.totales.estadoInstalacion'
                                                                ,'nombreSede':'gestion.totales.nombreInstitucionEd'
                                                                ,'matricula':'gestion.totales.matricula'
                                                                
                                                            ,'cod_servicio':'gestion.estado.cod_servicio'
                                                            ,'codDaneMuni':'gestion.estado.codDaneMuni'
                                                            ,'codCentroPoblado':'gestion.estado.codCentroPoblado'
                                                            ,'codDaneInstitucionEdu':'gestion.estado.codDaneInstitucionEdu'
                                                            ,'tipoSitio':'gestion.estado.tipoSitio'
                                                            ,'detalleSitio':'gestion.estado.detalleSitio'
                                                            ,'energia':'gestion.estado.energia'
                                                            ,'region':'gestion.estado.region'
                                                            ,'grupoDesc' :'gestion.estado.grupoDesc'
#                                                             ,'estadoInstalacion':'gestion.estado.estadoInstalacion'
                                                     
                                                     })
    
    conteo_totalesx['gestion.totales.nombreSede'] = conteo_totalesx['gestion.totales.nombreInstitucionEd']
    conteo_totalesx['gestion.estado.estadoInstalacion'] = conteo_totalesx['gestion.totales.estadoInstalacion']
    
    conteo_totalesx["gestion.fecha"] = conteo_totalesx["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    conteo_totalesx["gestion.hora"] = conteo_totalesx["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    conteo_totalesx["gestion.anyo"] = conteo_totalesx["gestion.fecha"].str[0:4]
    conteo_totalesx["gestion.mes"] = conteo_totalesx["gestion.fecha"].str[5:7]
    conteo_totalesx["gestion.dia"] = conteo_totalesx["gestion.fecha"].str[8:10]
    totalesx = conteo_totalesx
except KeyError as err:
    totalesx =pd.DataFrame([])
    print("hubo error ", err)
except:
    totalesx =pd.DataFrame([])
    print('conteo_incidentes o conteo_interacciones no existen')

In [ ]:
# 'gestion.totales.estadoInstalacion' in conteo_incidentes, \
# 'gestion.totales.estadoInstalacion' in conteo_interacciones,\
# 'gestion.totales.estadoInstalacion' in conteo_llamadas,\
# 'gestion.totales.estadoInstalacion' in conteo_correos,\
# 'gestion.totales.estadoInstalacion' in conteo_totalesx

# Despues del calculo la insercion de Tickets


In [ ]:
use_these_keys = ['gestion.totales.site_id'
                    ,'gestion.totales.total_tickets'
                    ,'gestion.totales.COD_ISO'
                    ,'gestion.totales.codDanesede'
                    ,'gestion.totales.nombre_centro_pob'
                    ,'gestion.totales.dptoGestion'
                    ,'gestion.totales.energiadesc'
                    ,'gestion.totales.muniGestion'
                    ,'gestion.totales.id_Beneficiario'
                    ,'gestion.totales.location'
                    , 'gestion.totales.DDA'
                    , 'gestion.totales.estadoInstalacion'
                    , 'gestion.totales.nombreInstitucionEd'
                    , 'gestion.totales.matricula'
                    , 'gestion.totales.estadoInstalacion'
#                   , 'gestion.totales.municipioPDET'
                    , 'gestion.fechaControl'
                    , 'gestion.fecha'
                    , 'gestion.anyo'
                    , 'gestion.mes'
                    , 'gestion.dia'
                    , 'gestion.hora'
                
                  , 'gestion.estado.cod_servicio'
                  , 'gestion.estado.codDaneMuni'                  
                  , 'gestion.estado.codCentroPoblado'
                  , 'gestion.estado.codDaneInstitucionEdu'
                  , 'gestion.estado.tipoSitio'
                  , 'gestion.estado.detalleSitio'
                  , 'gestion.estado.energia'
                  , 'gestion.estado.region'
                  , 'gestion.estado.grupoDesc'
                  , 'gestion.estado.estadoInstalacion'
                  
                    , '@timestamp']

#conteo_incidentes['gestion.totales.fecha_control'] = fecha_max_mintic

totalesx['@timestamp'] = now.isoformat()

In [9]:
# #$$$$$$$$$$$
# t =pd.DataFrame([])
# now = datetime.now()
# t['@timestamp'] = now.isoformat()
# t.shape

(0, 1)

In [ ]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'totales-tickets-'+str(document['gestion.totales.id_Beneficiario']) + '-' + str(document['gestion.fechaControl'])}",
                "_source": filterKeys(document),
            }
if totalesx.shape[0]!=0:
    salida = helpers.bulk(es, doc_generator(totalesx))
    print("Fecha: ", now,"- Totales tickets insertados en indice principal:",salida[0]) 

### Realizando inserción conteo de llamadas

In [ ]:
use_these_keys = ['gestion.totales.id_Beneficiario'
                    ,'gestion.totales.llamadas'
#                     ,'gestion.totales.nombreSede'
                    ,'gestion.totales.COD_ISO'
                    ,'gestion.totales.codDanesede'
                    ,'gestion.totales.nombre_centro_pob'
                    ,'gestion.totales.site_id'
                    ,'gestion.totales.dptoGestion'
                    ,'gestion.totales.energiadesc'
                    ,'gestion.totales.muniGestion'
                    ,'gestion.totales.location'
                    ,'gestion.totales.estadoInstalacion'
                    , 'gestion.fechaControl'
                    , 'gestion.fecha'
                    , 'gestion.anyo'
                    , 'gestion.mes'
                    , 'gestion.dia'
                    , 'gestion.hora'
                  , 'gestion.estado.cod_servicio'
                  , 'gestion.estado.codDaneMuni'                  
                  , 'gestion.estado.codCentroPoblado'
                  , 'gestion.estado.codDaneInstitucionEdu'
                  , 'gestion.estado.tipoSitio'
                  , 'gestion.estado.detalleSitio'
                  , 'gestion.estado.energia'
                  , 'gestion.estado.region'
                  , 'gestion.estado.grupoDesc'
                  , 'gestion.estado.estadoInstalacion'
                                      , '@timestamp']
#conteo_llamadas['gestion.totales.fecha_control'] = fecha_max_mintic2
conteo_llamadas['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'totales-llamadas-'+str(+document['gestion.totales.id_Beneficiario']) + '-' + str(document['gestion.fechaControl'])}",
                "_source": filterKeys(document),
            }

salida = helpers.bulk(es, doc_generator(conteo_llamadas))
print("Fecha: ", now,"- Llamadas insertadas en indice principal:",salida[0])

### Realizando inserción conteo de Correos

In [ ]:
use_these_keys = ['gestion.totales.id_Beneficiario'
                    ,'gestion.totales.correos'
                    ,'gestion.totales.nombreSede'
                    ,'gestion.totales.COD_ISO'
                    ,'gestion.totales.codDanesede'
                    ,'gestion.totales.nombre_centro_pob'
                    ,'gestion.totales.site_id'
                    ,'gestion.totales.dptoGestion'
                    ,'gestion.totales.energiadesc'
                    ,'gestion.totales.muniGestion'
                    ,'gestion.totales.location'
                    ,'gestion.totales.DDA'
                    ,'gestion.totales.estadoInstalacion'
                    ,'gestion.totales.nombreInstitucionEd'
                    ,'gestion.totales.matricula'
#                     ,'gestion.totales.municipioPDET'
                    ,'gestion.fechaControl'
                    , 'gestion.fecha'
                    , 'gestion.anyo'
                    , 'gestion.mes'
                    , 'gestion.dia'
                    , 'gestion.hora'
                  
                  , 'gestion.estado.cod_servicio'
                  , 'gestion.estado.codDaneMuni'                  
                  , 'gestion.estado.codCentroPoblado'
                  , 'gestion.estado.codDaneInstitucionEdu'
                  , 'gestion.estado.tipoSitio'
                  , 'gestion.estado.detalleSitio'
                  , 'gestion.estado.energia'
                  , 'gestion.estado.region'
                  , 'gestion.estado.grupoDesc'
                  , 'gestion.estado.estadoInstalacion'
                  
                    , '@timestamp']

#conteo_correos['gestion.totales.fecha_control'] = fecha_max_mintic2
conteo_correos['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'totales-correos-'+str(+document['gestion.totales.id_Beneficiario']) + '-' + str(document['gestion.fechaControl'])}",
                "_source": filterKeys(document),
            }

salida = helpers.bulk(es, doc_generator(conteo_correos))
print("Fecha: ", now,"- Correos insertados en indice principal:",salida[0])

### Realizando inserción conteo de incidentes

In [ ]:
use_these_keys = ['gestion.totales.site_id'
                    ,'gestion.totales.total_incidentes'
                    ,'gestion.totales.nombreSede'
                    ,'gestion.totales.COD_ISO'
                    ,'gestion.totales.codDanesede'
                    ,'gestion.totales.nombre_centro_pob'
                    ,'gestion.totales.dptoGestion'
                    ,'gestion.totales.energiadesc'
                    ,'gestion.totales.muniGestion'
                    ,'gestion.totales.id_Beneficiario'
                    ,'gestion.totales.location'
                    , 'gestion.totales.DDA'
                    , 'gestion.totales.estadoInstalacion'
                    , 'gestion.totales.nombreInstitucionEd'
                    , 'gestion.totales.matricula'
#                     , 'gestion.totales.municipioPDET'
                    , 'gestion.fechaControl'
                    , 'gestion.fecha'
                    , 'gestion.anyo'
                    , 'gestion.mes'
                    , 'gestion.dia'
                    , 'gestion.hora'
                  
                  , 'gestion.estado.cod_servicio'
                  , 'gestion.estado.codDaneMuni'                  
                  , 'gestion.estado.codCentroPoblado'
                  , 'gestion.estado.codDaneInstitucionEdu'
                  , 'gestion.estado.tipoSitio'
                  , 'gestion.estado.detalleSitio'
                  , 'gestion.estado.energia'
                  , 'gestion.estado.region'
                  , 'gestion.estado.grupoDesc'
                  , 'gestion.estado.estadoInstalacion'
                  
                    , '@timestamp']

#conteo_incidentes['gestion.totales.fecha_control'] = fecha_max_mintic

conteo_incidentes['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'totales-incidentes-'+str(document['gestion.totales.id_Beneficiario']) + '-' + str(document['gestion.fechaControl'])}",
                "_source": filterKeys(document),
            }

salida = helpers.bulk(es, doc_generator(conteo_incidentes))
print("Fecha: ", now,"- Incidentes insertados en indice principal:",salida[0])    


### Actualizando fecha de control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = "2021-05-01T00:00:00"

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_totales_incidentes',
        body = { 'jerarquia_gestion_totales_incidentes': 'gestion_totales_incidentes','gestion.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

In [ ]:
fecha_ejecucion2 = (datetime.strptime(fecha_max_mintic2, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(hours=24)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion2 = "2021-05-01T00:00:00"

if fecha_ejecucion2 > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion2 = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_totales_llamadas',
        body = { 'jerarquia_gestion_totales_llamadas': 'gestion_totales_llamadas','gestion.fechaControl' : fecha_ejecucion2}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion2)